In [2]:
import pandas as pd
import requests
import io
import urllib3
import ssl
from google.colab import drive


ssl._create_default_https_context = ssl._create_unverified_context
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Montando o drive
drive.mount('/content/drive')
PATH_DESTINO = '/content/drive/My Drive/Projeto GitHub/'
headers = {'User-Agent': 'Mozilla/5.0'}

# URLs dos dados de 2025
URLS = {
    "IMPORTACAO": "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_2025.csv",
    "EXPORTACAO": "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_2025.csv",
    "PAISES": "https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv",
    "NCM": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv"
}

def carregar_dados(url, encoding='utf-8'):
    response = requests.get(url, verify=False, headers=headers)
    return pd.read_csv(io.BytesIO(response.content), sep=';', encoding=encoding)


try:


    # Coleta
    df_imp_raw = carregar_dados(URLS["IMPORTACAO"])
    df_exp_raw = carregar_dados(URLS["EXPORTACAO"])
    df_aux_pais = carregar_dados(URLS["PAISES"], encoding='latin-1')
    df_aux_ncm = carregar_dados(URLS["NCM"], encoding='latin-1')

    # Processamento SC
    def processar_sc(df, tipo):
        df_sc = df[df['SG_UF_NCM'] == 'SC'].copy()
        df_sc['MOVIMENTACAO'] = tipo
        df_sc = pd.merge(df_sc, df_aux_pais[['CO_PAIS', 'NO_PAIS']], on='CO_PAIS', how='left')
        df_sc = pd.merge(df_sc, df_aux_ncm[['CO_NCM', 'NO_NCM_POR']], on='CO_NCM', how='left')

        return df_sc[['CO_ANO', 'CO_MES', 'MOVIMENTACAO', 'NO_PAIS', 'NO_NCM_POR', 'VL_FOB']]

    print("Filtrando dados de Santa Catarina...")
    df_imp_sc = processar_sc(df_imp_raw, 'Importação')
    df_exp_sc = processar_sc(df_exp_raw, 'Exportação')

    # Unificar e salvar
    df_final = pd.concat([df_imp_sc, df_exp_sc], ignore_index=True)

    nome_csv = 'dados_comex_sc_2025.csv'
    df_final.to_csv(PATH_DESTINO + nome_csv, index=False, sep=';', encoding='utf-8-sig')


    print(f"Total de registros de SC: {len(df_final)}")

except Exception as e:
    print(f"\nErro durante a execução: {e}")

Mounted at /content/drive
Iniciando coleta de dados... (isso pode levar uns instantes)
Baixando: https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_2025.csv
Baixando: https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_2025.csv
Baixando: https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv
Baixando: https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv
Filtrando dados de Santa Catarina...

--- SUCESSO! ---
Arquivo salvo em: /content/drive/My Drive/Projeto GitHub/dados_comex_sc_2025.csv
Total de registros de SC: 531675
